In [1]:
from fastai.vision import *

# Squash Classifier

I love winter squash. I typically hate summer squash. This classifier will help me make sure that I never accidentally purchas a zucchini squash when what I really want is an acorn squash.

In [9]:
types = ['butternut', 'pumpkin', 'acorn', 'zucchini', 'tromboncino']

In [10]:
# create directories for training data
for type in types:
    folder = type
    file = f'urls_{type}'
    path = Path('data/squash')
    dest = path/folder
    dest.mkdir(parents=True, exist_ok=True)

In [11]:
# make sure all my directories are in place
path.ls()

[PosixPath('data/squash/pumpkin'),
 PosixPath('data/squash/tromboncino'),
 PosixPath('data/squash/.ipynb_checkpoints'),
 PosixPath('data/squash/acorn'),
 PosixPath('data/squash/zucchini'),
 PosixPath('data/squash/butternut')]

In [13]:
for type in types:
    download_images(path/f'{type}.txt', path/f'{type}/')

Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '

Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '

Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '': No schema supplied. Perhaps you meant http://?
Error  Invalid URL '

In [18]:
for type in types:
    print(type)
    verify_images(path/type, delete=True, max_size=500)

butternut


pumpkin


acorn


zucchini


tromboncino


In [17]:
np.random.seed(333)
data = ImageDataBunch(.from_folder(path, train='.', valid_pct=0.2,
                                  ds_tfms=))

[PosixPath('data/squash/pumpkin'),
 PosixPath('data/squash/pumpkin.txt'),
 PosixPath('data/squash/tromboncino.txt'),
 PosixPath('data/squash/acorn.txt'),
 PosixPath('data/squash/tromboncino'),
 PosixPath('data/squash/.ipynb_checkpoints'),
 PosixPath('data/squash/acorn'),
 PosixPath('data/squash/butternut.txt'),
 PosixPath('data/squash/zucchini.txt'),
 PosixPath('data/squash/zucchini'),
 PosixPath('data/squash/butternut')]

In [22]:
doc(ImageDataBunch)

class ImageDataBunch [source] [test] ImageDataBunch ( train_dl : DataLoader , valid_dl : DataLoader , fix_dl : DataLoader = None , test_dl : Optional [ DataLoader ]= None , device : device = None , dl_tfms : Optional [ Collection [ Callable ]]= None , path : PathOrStr = '.' , collate_fn : Callable = 'data_collate' , no_check : bool = False ) :: DataBunch 
 
 × No tests found for ImageDataBunch . To contribute a test please refer to this guide and this discussion . DataBunch suitable for computer vision. 
 Show in docs

Init signature:
ImageDataBunch(
    train_dl: torch.utils.data.dataloader.DataLoader,
    valid_dl: torch.utils.data.dataloader.DataLoader,
    fix_dl: torch.utils.data.dataloader.DataLoader = None,
    test_dl: Union[torch.utils.data.dataloader.DataLoader, NoneType] = None,
    device: torch.device = None,
    dl_tfms: Union[Collection[Callable], NoneType] = None,
    path: Union[pathlib.Path, str] = '.',
    collate_fn: Callable = <function data_collate at 0x7ff39529e5f0>,
    no_check: bool = False,
)
Source:        
class ImageDataBunch(DataBunch):
    "DataBunch suitable for computer vision."
    _square_show = True

    @classmethod
    def create_from_ll(cls, lls:LabelLists, bs:int=64, val_bs:int=None, ds_tfms:Optional[TfmList]=None,
                num_workers:int=defaults.cpus, dl_tfms:Optional[Collection[Callable]]=None, device:torch.device=None,
                test:Optional[PathOrStr]=None, collate_fn:Callable=data_collate, size:int=None, no_check:bool=False,
              